# Scraping specific information from detail pages using Selenium

The objective of this section is to scrape specific information from the detail pages of all Chinese novels on the Wuxia World website. The key information includes the title, author, genres, rating, number of chapters, number of reviews, and details of the reviews.

## 1.Installing Selenium and Browser Driver

In this case, we chose to use Selenium because the Wuxia World website has implemented an anti-scraping mechanism that requires browser simulation to obtain complete data. For installing and documentation, please refer to https://selenium-python.readthedocs.io/index.html

Note: Please make sure to install the necessary libraries such as Selenium and BeautifulSoup before running the code. You can use pip, the Python package installer, to install these libraries. To install them, open your command prompt or terminal and enter the following commands:

pip install selenium;
pip install beautifulsoup4

In [89]:
# Import the required libraries
import csv
import os
import re
import time
from lxml import etree
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Here, we take ChromeDriver as an example. Please follow the setup instruction https://sites.google.com/chromium.org/driver/getting-started

## 2. Analyzing the webpage structure

Firstly, we need to browse the page structure of the target website and examine the web pages to determine their types.

Based on observation, we have noticed that all the Chinese novels are listed in alphabetical order in a table on the webpage. Each target novel's title corresponds to a hyperlink that leads to its detail page. In the table, only partial information, such as the novel's name and rating, is displayed. To access more detailed information, one needs to click on the hyperlink. Therefore, our overall web scraping approach is as follows:

(1).Obtain a complete list of links corresponding to all the novels;
(2).Iterate through the target link list to scrape specific information from each webpage and store it locally.

## 3. Obtaining the URL list

Using the developer tool, we discovered that the URL is contained within the "href" attribute of the novel's title. To obtain a complete list of URLs for the novels, we will follow the steps of locating the novel's title and extracting the "href" attribute value of the element.

Selenium provides various methods for locating elements, such as XPath, ID, and name. In this case, we will use XPath to locate the element. Once we have obtained the XPath of the corresponding element, we will use the syntax "@href" to select the element's attribute.

In [90]:
# Specify the path of the chromedriver and launch the browser
driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver')
# Navigate to the URL of wuxiaworld
driver.get('https://www.wuxiaworld.com/novels')

# Wait for 10 seconds for the page to load
time.sleep(10)
    
# Click the 'Chinese' checkbox to show only Chinese novels
driver.find_element(By.XPATH,'//*[@id="loading-container-replacement"]/div/div[1]/div[2]/div/div/div/div[1]/div/label[2]/span[2]').click()

# Wait for 10 seconds for the page to load
time.sleep(10)
    
# Scroll down the page multiple times to load all the novel data
for i in range(60):
    # Simulate the Page Down key press using ActionChains
    ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
    # Wait for a short time for the page to load
    time.sleep(0.2)
        
# Create an empty list to store all Chinese novels URLs
all_url_list = []

# Xpath locate the element
url_xpath = '//*[@id="loading-container-replacement"]/div/div[2]/div/div/div/div/div/div/div/div/div[2]/p/a/@href'

# Get the HTML source of the page
html = driver.page_source
# Parse the HTML source using the etree.HTML function
tree = etree.HTML(html)
# Extract the list of URLs using the XPath expression
url_list = tree.xpath(url_xpath)
# Extend the all_url_list with the extracted URL list
all_url_list.extend(url_list)
    

<ipython-input-90-c656d770cb62>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver')


## 4. Iterating through the URL list to scrape specific information from the detail pages

After obtaining the complete URL list of the novels, we need to iterate through the list to retrieve the details page of each novel. On this page, the key content we need to scrape includes:

(1)Title;
(2)Author;
(3)Genres;
(4)Rating;
(5)Number of chapters;
(6)Number of reviews;
(7)Details of the reviews.

Please note that the above content is the crucial information we need to extract from the details page of each novel.

### 4.1 Scraping the Title, Author, Genres, Rating, Number of chapters, and Number of reviews from the novel pages corresponding to each URL 

In [74]:
# Browser driver configuration and usage
# Create an instance of ChromeOptions to customize Chrome browser settings
CHROME_OPTIONS = webdriver.ChromeOptions()
# Specify the preference for displaying images
# 1 for displaying images, 2 for not displaying images. When images are not needed to be crawled, they can be set to not load images to save time.
prefs = {"profile.managed_default_content_settings.images":2}   
# Add the image display preference to the Chrome options
CHROME_OPTIONS.add_experimental_option("prefs", prefs)
# Create a Chrome WebDriver instance with the specified driver executable path and options
driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver', options=CHROME_OPTIONS)

# Xpath locating for basic novel information
title_xpath = '//*[@id="loading-container-replacement"]/div/div[1]/div/div/div[2]/div[1]/div[2]/h1'
author_xpath = '//*[@id="loading-container-replacement"]/div/div[1]/div/div/div[2]/div[3]/div[1]/div[2]'
genres_xpath = '//*[@id="full-width-tabpanel-0"]/div/div[1]/div[2]/div/a/div/div'
rating_xpath = '//*[@id="loading-container-replacement"]/div/div[1]/div/div/div[2]/div[2]/div/span/span'
chapters_xpath = '//*[@id="full-width-tabpanel-0"]/div/div[1]/div[1]/div[1]/div[2]'
reviews_xpath = '//*[@id="loading-container-replacement"]/div/div[1]/div/div/div[2]/div[2]/div/div/span'

# Create an empty list to store all the title，author,genres,rating,chapters and reviews 
all_title = []
all_author = []
all_genres = []
all_rating = []
all_chapters = []
all_reviews = []

# Iterate through the list of URLs
for url_item in all_url_list:
    url = 'https://www.wuxiaworld.com' + url_item
    driver.get(url) 
    time.sleep(5)
    # Parse the webpage
    html = driver.page_source
    tree = etree.HTML(html)
    
    # Extract the text nodes of novel basic information
    title_list = tree.xpath(title_xpath + '/text()')
    all_title.append(title_list[0])
    
    author_list = tree.xpath(author_xpath + '/text()')
    all_author.append(author_list[0])
    
    genres_list = tree.xpath(genres_xpath+ '/text()')
    content = ';'.join(genres_list)  # There may be multiple genres, concatenate them with a semicolon
    all_genres.append(content)
    
    rating_list = tree.xpath(rating_xpath + '/text()')
    all_rating.append(rating_list[0])
    
    chapters_list = tree.xpath(chapters_xpath + '/text()')
    all_chapters.append(chapters_list[0])
    
    reviews_list = tree.xpath(reviews_xpath + '/text()')
    all_reviews.append(reviews_list[0])

# Specify the file path and open the CSV file in write mode with UTF-8 encoding    
with open('data_list.csv', 'w',encoding='utf-8',newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.writer(csvfile)
    # Write the header row with column names
    writer.writerow(['title','author','genres','rating','chapters','reviews','url'])
    # Iterate through each URL and corresponding data, and write them as rows in the CSV file
    for url_item, row in zip(all_url_list, zip(all_title, all_author, all_genres, all_rating, all_chapters, all_reviews)):
        # Convert the row elements to a list and append the URL item to the end
        writer.writerow(list(row) + [f'https://www.wuxiaworld.com{url_item}'])

# Close the browser driver
driver.quit()


<ipython-input-74-54f9fccd8d19>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver', options=CHROME_OPTIONS)


### 4.2 Scraping the details of the reviews from the novel pages corresponding to each URL

In [92]:
# Specify the path of the chromedriver and launch the browser
driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver')
# Open the novel page
driver.get('https://www.wuxiaworld.com/novel/rmji')

# Wait for 10 seconds for the page to load
time.sleep(10)

# Replace the generic view_all path
driver.find_element(By.XPATH,'//*[@id="full-width-tabpanel-0"]/div/div[3]/div[2]/div[2]/div/div[2]/div/span').click()

# Wait for 10 seconds for the page to load
time.sleep(10)

# Create an empty list to store all the detail reviews
detail_reviews_list= []

# Get the reviews on the first page
html = driver.page_source # Get the HTML source code of the current page
soup = BeautifulSoup(html, "html.parser") # Create a BeautifulSoup object to parse the HTML
data = soup.find_all('div', class_="absolute top-0 -z-10 line-clamp-1 font-set-r15-h150 text-gray-t1 sm2:font-set-r16-h150")
# Find all the review elements with the specified class
# Note: The class represents the CSS styles applied to the review elements

data = data[3:] # Remove the first three reviews from the list of review elements
detail_reviews_list.extend(data) # Add the remaining reviews to the detail_reviews_list

# Use a set to keep track of already retrieved reviews
seen_reviews = set([r.get_text(strip=True) for r in detail_reviews_list])

# Get the reviews from the next page
next_page=driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div/div/div/div[2]/div[3]/nav/ul/li[last()-0]/button')

# Keep clicking the next page button until it's disabled
while next_page.is_enabled():
    next_page.click()
    time.sleep(5)
    html = driver.page_source # Get the HTML source code of the current page
    soup = BeautifulSoup(html, "html.parser") # Create a BeautifulSoup object to parse the HTML
    data = soup.find_all('div', class_="absolute top-0 -z-10 line-clamp-1 font-set-r15-h150 text-gray-t1 sm2:font-set-r16-h150")
    # Find all the review elements with the specified class
    # Note: The class represents the CSS styles applied to the review elements
    
    # Add the unseen reviews to the detail_reviews_list
    for review in data:
        if review.get_text(strip=True) not in seen_reviews:
            detail_reviews_list.append(review)
            seen_reviews.add(review.get_text(strip=True))
    # Find the next page button and wait until it is present
    next_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[2]/div[3]/div/div/div/div[2]/div[3]/nav/ul/li[last()-0]/button')))

# Open a text file named 'detail_reviews.txt' in write mode, using UTF-8 encoding
with open('detail_reviews.txt', 'w', encoding='utf-8') as f:
    # Iterate through each detail_review in the detail_reviews_list
    for detail_review in detail_reviews_list:
        # Write the stripped text of the detail_review to the file
        f.write(detail_review.get_text(strip=True))
        # Write two newline characters to create a blank line between reviews
        f.write('\n\n')

# Close the browser driver        
driver.quit()

<ipython-input-92-6a0aa88beaf5>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver')


# Installing, Importing and Preprocessing

Due to the huge amount of data (about 9000 reviews), the loading time is too long, so the reviews of the top 5 novels in rating (about 500 reviews) were chosen as the corpus in this assignment to improve the efficiency.

In [2]:
# Install the langdetect library
!pip install langdetect

In [1]:
#Install the spaCy library
!pip install spacy==3.5.2

     |████████████████████████████████| 6.8 MB 2.2 MB/s eta 0:00:01


In [11]:
# Import the required libraries
import csv
import os
import re
from langdetect import detect

In the preprocessing stage, keep only English reviews, remove non-letter, number, space and punctuation special characters. Additionally, all reviews were converted to lowercase. 

In [12]:
# Use relative path containing the files for reproducibility.
corpora_folder_path = './corpora'

# List to store filenames, original reviews, and processed reviews
file_data = []

# Regular expressions to match characters that are not letters, numbers, spaces and punctuation marks
pattern = re.compile('[^a-zA-Z0-9\s.,?!]')

# Iterate through files in the specified folder
for file_name in os.listdir(corpora_folder_path):
    file_path = os.path.join(corpora_folder_path, file_name)
    
    # Ensure the current item is a file, not a folder
    if os.path.isfile(file_path):
        # Extract filenames and original reviews
        filename = os.path.basename(file_path)
        reviews_list = []
        
        # Read the file and extract reviews
        with open(file_path, 'r', encoding='latin-1') as file:
            review = ''
            for line in file:
                # Check for empty lines, add the previous review to the list, and clear the current review
                if not line.strip():
                    if review:
                        reviews_list.append(review.strip())
                        review = ''
                else:
                    review += line

            # Process the last review (in case the file ends without an empty line)
            if review:
                reviews_list.append(review.strip())
        
        # Keep only English reviews using langdetect
        english_reviews = [review for review in reviews_list if detect(review) == 'en']
        
        #Use regular expressions to remove non-letter, number, space and punctuation special characters
        cleaned_reviews = [pattern.sub('', review) for review in english_reviews]
        
        # Unify reviews to lowercase using the lower() method
        cleaned_reviews_lower = [review.lower() for review in cleaned_reviews]
        
        # Extend the file_data list with tuples containing filename, original review, and cleaned review
        file_data.extend([(filename, document, text) for document, text in zip(reviews_list, cleaned_reviews_lower)])


In [13]:
# Write the cleaned reviews to a CSV file using relative path
with open(csv_output_path, 'w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Write column names to the CSV file
    csv_writer.writerow(['filename', 'document', 'text'])
    
    # Write filenames, original reviews and processed reviews
    csv_writer.writerows(file_data)


To improve the quality of the corpus, invalid reviews can also be removed manually after removing non-letter, number, space and punctuation special characters.

# Text Enrichment with spaCy

## Creating Doc Objects

In [5]:
pip install numpy==1.21

     |████████████████████████████████| 16.9 MB 597 kB/s eta 0:00:01     |█████████████████████████       | 13.2 MB 4.7 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Install English language model
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 5.6 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
# Import the required libraries
import os
import pandas as pd
import spacy

In [18]:
# Load the spaCy English language model
nlp = spacy.load('en_core_web_sm')

# Define your process_text function
def process_text(text):
    # Apply spaCy NLP pipeline to each document
    doc = nlp(text)
    return doc

# Get the current working directory
current_working_directory = os.getcwd()

# Specify a csv file using relative path
csv_filename = 'reviews_output.csv'
csv_file_path = os.path.join(current_working_directory, csv_filename)

# Read a csv file into DataFrame
df = pd.read_csv(csv_file_path)

# Apply the process_text function to the 'text' column
df['Doc'] = df['text'].apply(process_text)

## Text Reduction

### Tokenization 

In [19]:
# Define a function to retrieve tokens from a doc object
def get_token(doc):
    return [(token.text) for token in doc]

In [20]:
# Run the token retrieval function on the doc objects in the dataframe
df['Tokens'] = df['Doc'].apply(get_token)
df.head()

,filename,document,text,Doc,Tokens
0,reviews Coiling Dragon.txt,Where do I even start on this novel?it is famo...,where do i even start on this novel?it is famo...,"(where, do, i, even, start, on, this, novel?it...","[where, do, i, even, start, on, this, novel?it..."
1,reviews Coiling Dragon.txt,Like many this was the first c-novel i ever re...,like many this was the first cnovel i ever rea...,"(like, many, this, was, the, first, cnovel, i,...","[like, many, this, was, the, first, cnovel, i,..."
2,reviews Coiling Dragon.txt,"It's Coiling Dragon, obviously you should read...","its coiling dragon, obviously you should read ...","(its, coiling, dragon, ,, obviously, you, shou...","[its, coiling, dragon, ,, obviously, you, shou..."
3,reviews Coiling Dragon.txt,This is the one that got me into the entire ge...,this is the one that got me into the entire ge...,"(this, is, the, one, that, got, me, into, the,...","[this, is, the, one, that, got, me, into, the,..."
4,reviews Coiling Dragon.txt,Not a single fault of the series comes to mind...,not a single fault of the series comes to mind...,"(not, a, single, fault, of, the, series, comes...","[not, a, single, fault, of, the, series, comes..."


### Lemmatization 

In [21]:
# Define a function to retrieve lemmas from a doc object
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

# Run the lemma retrieval function on the doc objects in the dataframe
df['Lemmas'] = df['Doc'].apply(get_lemma)
df.head()

,filename,document,text,Doc,Tokens,Lemmas
0,reviews Coiling Dragon.txt,Where do I even start on this novel?it is famo...,where do i even start on this novel?it is famo...,"(where, do, i, even, start, on, this, novel?it...","[where, do, i, even, start, on, this, novel?it...","[where, do, I, even, start, on, this, novel?it..."
1,reviews Coiling Dragon.txt,Like many this was the first c-novel i ever re...,like many this was the first cnovel i ever rea...,"(like, many, this, was, the, first, cnovel, i,...","[like, many, this, was, the, first, cnovel, i,...","[like, many, this, be, the, first, cnovel, I, ..."
2,reviews Coiling Dragon.txt,"It's Coiling Dragon, obviously you should read...","its coiling dragon, obviously you should read ...","(its, coiling, dragon, ,, obviously, you, shou...","[its, coiling, dragon, ,, obviously, you, shou...","[its, coil, dragon, ,, obviously, you, should,..."
3,reviews Coiling Dragon.txt,This is the one that got me into the entire ge...,this is the one that got me into the entire ge...,"(this, is, the, one, that, got, me, into, the,...","[this, is, the, one, that, got, me, into, the,...","[this, be, the, one, that, get, I, into, the, ..."
4,reviews Coiling Dragon.txt,Not a single fault of the series comes to mind...,not a single fault of the series comes to mind...,"(not, a, single, fault, of, the, series, comes...","[not, a, single, fault, of, the, series, comes...","[not, a, single, fault, of, the, series, come,..."


## Text Annotation 

### Part of Speech Tagging

In [23]:
# Define a function to retrieve lemmas from a doc object
def get_pos(doc):
    #Return the coarse- and fine-grained part of speech text for each token in the doc
    return [(token.pos_, token.tag_) for token in doc]

# Define a function to retrieve parts of speech from a doc object
df['POS'] = df['Doc'].apply(get_pos)
# Drop the 'Doc' column
df = df.drop('Doc', axis=1)
# Display the DataFrame
df.head()

,filename,document,text,Tokens,Lemmas,POS
0,reviews Coiling Dragon.txt,Where do I even start on this novel?it is famo...,where do i even start on this novel?it is famo...,"[where, do, i, even, start, on, this, novel?it...","[where, do, I, even, start, on, this, novel?it...","[(SCONJ, WRB), (AUX, VBP), (PRON, PRP), (ADV, ..."
1,reviews Coiling Dragon.txt,Like many this was the first c-novel i ever re...,like many this was the first cnovel i ever rea...,"[like, many, this, was, the, first, cnovel, i,...","[like, many, this, be, the, first, cnovel, I, ...","[(INTJ, UH), (ADJ, JJ), (PRON, DT), (AUX, VBD)..."
2,reviews Coiling Dragon.txt,"It's Coiling Dragon, obviously you should read...","its coiling dragon, obviously you should read ...","[its, coiling, dragon, ,, obviously, you, shou...","[its, coil, dragon, ,, obviously, you, should,...","[(PRON, PRP$), (VERB, VBG), (NOUN, NN), (PUNCT..."
3,reviews Coiling Dragon.txt,This is the one that got me into the entire ge...,this is the one that got me into the entire ge...,"[this, is, the, one, that, got, me, into, the,...","[this, be, the, one, that, get, I, into, the, ...","[(PRON, DT), (AUX, VBZ), (DET, DT), (NOUN, NN)..."
4,reviews Coiling Dragon.txt,Not a single fault of the series comes to mind...,not a single fault of the series comes to mind...,"[not, a, single, fault, of, the, series, comes...","[not, a, single, fault, of, the, series, come,...","[(PART, RB), (DET, DT), (ADJ, JJ), (NOUN, NN),..."


## Download Enriched Dataset

In [24]:
# Use this step only to save  csv to your computer's working directory
df.to_csv('reviews_with_spaCy_tags.csv')